In [1]:
%matplotlib notebook
from beam_tomo import *
plt.rcParams['figure.max_open_warning'] = 50


# Previous work on beam tomography 

Link for report with previous beam tomography:(link)[https://docs.google.com/document/d/1uOGDw6qHZKiGV28OnOs043JRSdri8LaaIsc8oYo4Qxg/edit#bookmark=id.4wuwfttv8kr]


# Description of measurements 

1. Acquisition #1 - 17.29.50 tomography.pkl (z from 14.6 to 16.6, PWM = 7%, exp = 1/40 000, 25us)
2. Acquisition #2 - 17.44.35 tomography.pkl (z from 14.6 to 17.6, PWM = 7%, exp = 1/40 000, 25us)

3. Acquisition #3 - 17.50.51 tomography.pkl (z from 14.6 to 18.6, PWM = 8%, exp = 1/40 000, 25us) - seems decoupled

4. Acquisition #4 - (re-coupled) - 17.56.48 tomography.pkl (z from 14.7 to 18.7)
5. Acquisition #5 - 18.02.48 tomography.pkl -same as acquisition 4, after rechecking coupling;


# 1. List and load measurements 

In [2]:
filenames = [item for item in os.listdir() if ".pkl" in item]
filenames

filename = filenames[4]
print("Filename: ", filename)

#load data
tomo = Tomography(filename, (3,10))
tomo.load_data()

Filename:  17.44.35 tomography.pkl
Loaded
Tomography measurement:

        - Filename = 17.44.35 tomography.pkl

        - Number of beam rows = 3

        - Number of beam cols = 10

        - Z-spacing: 0.100mm

        - Number of cross sections: 30
        


# 2. Visualise cross sections 

In [ ]:
#load cross sections
for cross_sect_i in tomo.cross_sect_l:
    if cross_sect_i.image_rot is not None:
        cross_sect_i.show_cross_section(rotated = True)
    else:
        cross_sect_i.show_cross_section(rotated = False)

# 3. Find rotation angle and spacing

In [3]:
tomo.find_rot_spacing(angle_step = 1.0)

Extracting rotation angle for the lowest z cross section.


100%|███████████████████████████████████████████| 90/90 [00:18<00:00,  4.74it/s]


Optimal rotation angle = 47.00deg
Extracting the grid spacing
Average spacing [px] between beams = 145.25
Updating the rotation angle and rotated image for each cross section.


100%|███████████████████████████████████████████| 30/30 [00:02<00:00, 11.94it/s]


## 

# 4. Extract beam coord of first layer --> tomo.coord_init()

In [4]:
#tomo.coord_init()
tomo.init_coord()

Coordinates of beam in first layer were determined.


In [5]:
tomo.beam_l[0][0].beam_coord_l

[array([656.        , 365.        ,  14.69997668])]

# 5. Extract single beam coords across cross-section

In [8]:
#tomo.complete_coord()

for id_z in range(tomo.n_sections-1):
    roi_width = 100
    #for an arbitrary beam, extract the coords in the cross section z_1
    id_x = 0
    id_y = 0
    

    #beam info in cross section i
    beam_i = tomo.beam_l[id_x][id_y]
    coord_x, coord_y, coord_z = tomo.beam_l[id_x][id_y].beam_coord_l[id_z]
    image_i = tomo.cross_sect_l[id_z].image_rot
    roi_i = image_i[int(coord_x-roi_width/2.): int(coord_x+roi_width/2.),
                    int(coord_y-roi_width/2.): int(coord_y+roi_width/2.)]
    if id_z == 0:
        beam_i.roi_l.append(roi_i)


    #beam info in cross section i+1
    image_ip1 = tomo.cross_sect_l[id_z+1].image_rot
    roi_ip1 = image_ip1[int(coord_x-roi_width/2.): int(coord_x+roi_width/2.),
                    int(coord_y-roi_width/2.): int(coord_y+roi_width/2.)]

    #establish new coordinates v1
    peak_arr = feature.peak_local_max(roi_ip1, num_peaks = 1, min_distance=1)
    print(peak_arr)
    
    coord_x = int(peak_arr[0][0] + coord_x-roi_width/2.)
    coord_y = int(peak_arr[0][1] + coord_y-roi_width/2.)
    coord_z = tomo.cross_sect_z_l[id_z+1]
    
    #establish new coordintes v2
    #coord_roi_y, coord_roi_x = find_centroid(roi_ip1)
    #coord_x = coord_roi_x + int(coord_x-roi_width/2.)
    #coord_y = coord_roi_y + int(coord_y-roi_width/2.)
    #coord_z = tomo.cross_sect_z_l[id_z+1]
    
    #establish new coordinates v3

    #update beam_i information (roi_l and beam_coord_l)
    roi_ip1 = image_ip1[int(coord_x-roi_width/2.): int(coord_x+roi_width/2.),
                    int(coord_y-roi_width/2.): int(coord_y+roi_width/2.)]
    beam_i.roi_l.append(roi_ip1)
    beam_i.beam_coord_l.append(np.array([coord_x, coord_y, coord_z]))


#for beam_i, in cross section z+1, define ROI from coordinates in cross section z
#cross_sect_i = tomo.cross_sect_l[id_z+1]
#f, ax = cross_sect_i.show_cross_section()
#rect = Rectangle((coord_x-roi_width/2., coord_y-roi_width/2), roi_width, roi_width, linewidth=2, edgecolor='r', facecolor='none')
#ax.add_patch(rect)



#for beam_i, in cross section z+1, re-evaluate the position of the centroid


[[49 58]]
[[48 56]]
[[49 51]]
[[47 57]]
[[45 49]]
[[50 54]]
[[46 57]]
[[42 48]]
[[50 62]]
[[45 57]]
[[45 54]]
[[49 49]]
[[36 48]]
[[56 60]]
[[48 50]]
[[42 52]]
[[40 48]]
[[46 54]]
[[54 71]]
[[39 37]]
[[98 28]]
[[58 45]]
[[97 48]]
[[59 43]]
[[52 48]]
[[50 59]]
[[50  4]]
[[44  3]]
[[52 67]]


In [ ]:
tomo.beam_l[0][0].beam_coord_l

In [ ]:
for i in range(tomo.n_sections):
    roi_i = beam_i.roi_l[i]
    plt.figure()
    plt.title(f"z_id = {i:.0f},z = {tomo.cross_sect_z_l[i]:.2f}mm")
    plt.imshow(roi_i, origin="lower")
    plt.scatter(x = 50, y = 50, color = "red", s=2)
    

In [ ]:
for i in range(tomo.n_sections):
    plt.figure()
    plt.title(f"z_id = {i:.0f},z = {tomo.cross_sect_z_l[i]:.2f}mm")
    plt.imshow(tomo.cross_sect_l[i].image_rot, origin="lower")
    x, y, z = beam_i.beam_coord_l[i]
    plt.scatter(y,x, color = "red", s = 1)

In [ ]:
x_arr = np.array(beam_i.beam_coord_l)[:,0]
y_arr = np.array(beam_i.beam_coord_l)[:,1]

limit = 15
plt.scatter(x_arr[:limit],y_arr[:limit], )

In [ ]:
id_z = 0
tomo.beam_l[id_x][id_y].beam_coord_l[id_z]

In [ ]:
beam_i.beam_coord_l

In [ ]:
plt.imshow(roi_ip1)

# 6. Extract beam ROIs for each cross section

# 7. Extract beam parameters (tilt_x, tilt_y, div_x, div_y) 

# 8. Visualise cross sections with location of beams and ROIs

# 9. Visualise single beam (3D)

In [ ]:
tomo.find_beams()

## Visualise first layer with ROI

In [ ]:
time_start = time.time()

#call tomo.coord_init()
exp_num_peaks = tomo.shape[0]*tomo.shape[1]
cross_i = tomo.cross_sect_l[0]
spacing = int(cross_i.spacing_px)
image_i = tomo.cross_sect_l[0].image_rot

peak_arr = feature.peak_local_max(image_i, num_peaks = exp_num_peaks, min_distance = int(spacing*0.9))

#pass info to cross section
#tomo.cross_sect_l[0].beam_coord_l = peak_arr

#pass info to each beam in tomo.beam_l
#for beam_i in tomo.beam_l:
#    beam_i.beam_coord_l = peak_arr

#time_stop = time.time()
#print("The task took", time_stop-time_start, "s to complete.")

In [ ]:
a = list()
peak_arr[3]
a.append(peak_arr[3])

In [ ]:
tomo.cross_sect_l[0].beam_coord_l = peak_arr

In [ ]:
peak_arr = [[peak_arr[i,j] for j in range(tomo.shape[1])] for i in range(tomo.shape[0])]

In [ ]:
f, ax = tomo.cross_sect_l[0].show_image(rotated=True)

In [ ]:
peak_arr.shape

In [ ]:
peak_arr.reshape(tomo.shape[0], tomo.shape[1], -1)

In [ ]:
tomo.cross_sect_l[0].show_image()

In [ ]:
#label beams - line_id, col_id
kmeans_line = KMeans(n_clusters = 3)
kmeans_line.fit(peak_sorted_arr[:,0].reshape(-1,1))
line_id = kmeans_line.labels_

In [ ]:
plt.imshow(tomo.cross_sect_l[8].image_rot)

## Beam and cross section classes

## Collect cross sections of the same beam for different z